In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

57344/57026 [==============================] - 0s 1us/step


In [6]:
print('Training data:', train_data.shape)
print('Test data: ', test_data.shape)
print('Training sample: ',train_data[0])
print('Training target sample: ',  train_targets[0])

Training data: (404, 13)
Test data:  (102, 13)
Training sample:  [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
Training target sample:  15.2


prepare the data<br>
feature normalization<br>
substract the mean of the feature from each feature and divide each result by the standard deviation

In [7]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [8]:
from keras import models
from keras import layers

In [9]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

K-fold validation

In [11]:
k=4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('Processing fold {}'.format(i))
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[: i * num_val_samples], train_data[(i+1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[: i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
    

Processing fold 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Processing fold 1
Processing fold 2
Processing fold 3


In [12]:
print('all_scores is: ', all_scores)
print('mean all scores: ', np.mean(all_scores))

all_scores is:  [2.1258119450937403, 2.123235086403271, 2.9499799851143713, 2.359854839815952]
mean all scores:  2.3897204641068335


In [13]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

102/102 [==============================] - 0s 489us/step


In [14]:
test_mae_score

3.0284980325137867